# Caltech STEM Machine Learning Model

In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.7.0'

'2.7.0'

In [10]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /private/var/folders/6d/z_kcw4m17579w78q7fzm6wh00000gr/T/pip-req-build-hcelqo1h
  Running command git clone -q https://github.com/tensorflow/docs /private/var/folders/6d/z_kcw4m17579w78q7fzm6wh00000gr/T/pip-req-build-hcelqo1h
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
     |████████████████████████████████| 192 kB 3.2 MB/s eta 0:00:01
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=169380 sha256=b76aed599773090651759734468bb6e634e53b423d92f54429a8bac569104c30
  Stored in directory: /private/var/folders/6d/z_kcw4m17579w78q7fzm6wh00000gr/T/pip-ephem-wheel-cache-24rogt9s/wheels/3b/ee/a2/ab4d36a9a4af495bcb936f3e849d4b497b65fa40548a68d6c3
Successfully built tensorflow-docs
You should consider upgrading via the '/Users/srinivasan/Documents/Projects/sm-caltech-stem-machine-learning/venv/bin/python -m pip install --upgrade pip' command.
  Cloning https://github.com/tensorflow/docs to /pr

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers